In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

import json


from sklearn.metrics import roc_curve, auc, precision_recall_curve


from sklearn.metrics import (
    matthews_corrcoef,
    balanced_accuracy_score,
    f1_score,
    average_precision_score,
)
from sklearn.metrics import precision_score, recall_score

In [2]:
drugs_df = pd.read_csv("../processed_data/drug_fingerprints.csv", index_col=0)
drugs_df

,MACCS_fp,morgan_512_fp,morgan_1024_fp,pubchem_fp
drug,,,,
5-Fluorocytosine,0000000000000000000000000000000000000110001100...,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,1101110001100000000110001100100001000000000000...
Amikacin,0000000000000000000000000000000000000000000000...,0100000000000100000001000000001001010000000000...,0100000000000100000000000000001000010000000000...,1101110001111100000111101110111100000000000000...
Amoxicillin,0000000010010000000100000000000000001000000000...,0100010000000000001000000000000001001000000000...,0100010000000000001000000000000001001000000000...,1101110001111000000111101100111000000000000100...
Amphotericin B,0000000000000000000000000000000000000000000000...,0100000000000000000001000010000001011011100000...,0100000000000000000000000010000001011000100000...,1101110001111100000111111000111110000000000000...
Ampicillin,0000000010010000000100000000000000001000000000...,0100010000000000001000000000000001001000000001...,0100010000000000001000000000000001001000000000...,1101110001111000000111101100111000000000000100...
...,...,...,...,...
Ticarcillin,0000000010010000000100000000000000001000000000...,0100010000000010001000000000000001001000100000...,0100010000000010001000000000000001001000000000...,1101110001111000000111001100111000000000000110...
Tigecycline,0000000000000000000000000010000000000000000000...,0000000100000100000000000000010001001100011001...,0000000000000000000000000000000001001100010000...,1101110001111100000111101110111100000000000000...
Tobramycin,0000000000000000000000000000000000000000000000...,0000000000010000000001000000000000010000000000...,0000000000010000000000000000000000010000000000...,1101110001111100000111101110111100000000000000...


In [4]:
long_table = pd.read_csv("../processed_data/DRIAMS_combined_long_table.csv")
long_table = long_table[long_table["drug"].isin(drugs_df.index)]

long_table

,species,sample_id,drug,response,dataset
0,Staphylococcus epidermidis,e9adf43d-679b-497c-9849-1fa214838dd3,Meropenem,1,A
1,Staphylococcus epidermidis,e9adf43d-679b-497c-9849-1fa214838dd3,Ciprofloxacin,1,A
2,Staphylococcus epidermidis,e9adf43d-679b-497c-9849-1fa214838dd3,Cefepime,1,A
3,Staphylococcus epidermidis,e9adf43d-679b-497c-9849-1fa214838dd3,Cotrimoxazole,0,A
4,Staphylococcus epidermidis,e9adf43d-679b-497c-9849-1fa214838dd3,Imipenem,1,A
...,...,...,...,...,...
652766,Staphylococcus aureus,08bc8410-51ec-46d7-ac7b-afba9e6ba2cd_3313,Linezolid,0,D
652767,Staphylococcus aureus,08bc8410-51ec-46d7-ac7b-afba9e6ba2cd_3313,Rifampicin,0,D
652768,Staphylococcus aureus,08bc8410-51ec-46d7-ac7b-afba9e6ba2cd_3313,Tetracycline,0,D
652769,Staphylococcus aureus,08bc8410-51ec-46d7-ac7b-afba9e6ba2cd_3313,Tigecycline,0,D


In [14]:
# folder_template = "outputs/ResAMR_DrugZeroShot/ZS_noCNN_emb512_DRIAMS-{}_drugs_zero_shot_sp0"
drug_zs_results = []
threshold = 0.5
folder = "outputs/ResAMR_DrugZeroShot/ZS_noCNN_emb512_results"

for fname in os.listdir(folder):
    if not fname.startswith("DRIAMS"):
        continue
        
    drug_name = fname.split("_")[1]
    dset = fname.split("_")[0].split("-")[1]
    
    
    test_set = pd.read_csv(os.path.join(folder, fname))
    
    
    response_classes = test_set["response"].values
    predictions = test_set["Predictions"].values
    
    predicted_classes = (predictions>=threshold).astype(int)
    
    metrics =  {
            "mcc": matthews_corrcoef(response_classes, predicted_classes),
            "balanced_accuracy": balanced_accuracy_score(
                response_classes, predicted_classes
            ),
            "f1": f1_score(response_classes, predicted_classes, zero_division=0),
            "AUPRC": average_precision_score(
                response_classes, predictions
            ),
            "precision": precision_score(
                response_classes, predicted_classes, zero_division=0
            ),
            "recall": recall_score(
                response_classes, predicted_classes, zero_division=0
            ),
        }

    
    metrics["dataset"] = dset
    metrics["drug"]=drug_name
    drug_zs_results.append(metrics)







drug_zs_results = pd.DataFrame(drug_zs_results)
# drug_zs_results.columns = ["dataset", "drug", "test_AUPRC", "n_test_samples", "n_resistant_test_samples"]
drug_zs_results = drug_zs_results.drop("drug", axis=1)

drug_zs_results

/home/gvisona/miniconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:879: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/gvisona/miniconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:879: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/gvisona/miniconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:879: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/gvisona/miniconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:879: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/home/gvisona/miniconda3/envs/ml_env/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:879: UserWarning: No positive class found in y_true, recall is set to one for all threshold

,mcc,balanced_accuracy,f1,AUPRC,precision,recall,dataset,drug
0,0.136483,0.713903,0.094675,0.076051,0.051282,0.615385,B,Rifampicin
1,0.249162,0.862681,0.160000,0.107776,0.087977,0.882353,D,Amikacin
2,0.188368,0.608541,0.256881,0.173411,0.222222,0.304348,B,Imipenem
3,0.430249,0.680129,0.554028,0.824316,0.927632,0.394958,B,Ampicillin
4,-0.185695,0.362069,0.000000,0.102678,0.000000,0.000000,B,Metronidazole
...,...,...,...,...,...,...,...,...
158,0.106661,0.558329,0.121212,0.048456,0.111111,0.133333,D,Meropenem
159,0.233856,0.616811,0.477342,0.821741,0.909789,0.323550,A,Penicillin
160,0.000000,0.851852,0.000000,-0.000000,0.000000,0.000000,D,Telithromycin
161,0.100147,0.550468,0.522801,0.490443,0.442759,0.638171,C,Fosfomycin


In [21]:

mean_df = drug_zs_results.groupby("dataset").mean()
mean_df.columns = [c+"_average" for c in mean_df.columns]
std_df = drug_zs_results.groupby("dataset").std()
std_df.columns = [c+"_std" for c in std_df.columns]
std_df
joined_df = pd.merge(mean_df, std_df, left_index=True, right_index=True)
joined_df.columns = ["test_" + c for c in joined_df.columns]
joined_df
# mean_df

,test_mcc_average,test_balanced_accuracy_average,test_f1_average,test_AUPRC_average,test_precision_average,test_recall_average,test_mcc_std,test_balanced_accuracy_std,test_f1_std,test_AUPRC_std,test_precision_std,test_recall_std
dataset,,,,,,,,,,,,
A,0.281093,0.652252,0.379570,0.472825,0.443204,0.439870,0.240298,0.128655,0.255085,0.292644,0.320742,0.287945
B,0.353169,0.705559,0.397460,0.473135,0.510088,0.443787,0.279759,0.152954,0.277678,0.314272,0.363517,0.309523
C,0.174801,0.662351,0.251471,0.343128,0.315627,0.263973,0.281546,0.254129,0.298361,0.347088,0.362205,0.311327
D,0.198087,0.658522,0.283299,0.360113,0.338227,0.390202,0.217081,0.162238,0.227813,0.306362,0.313166,0.295077


In [23]:
metrics_order = ["mcc", "f1", "precision", "recall", "AUPRC", "balanced_accuracy"]
cols = []
for m in metrics_order:
    cols.append(f"test_{m}_average")
    cols.append(f"test_{m}_std")
    
joined_df = joined_df[cols]

In [24]:
joined_df

,test_mcc_average,test_mcc_std,test_f1_average,test_f1_std,test_precision_average,test_precision_std,test_recall_average,test_recall_std,test_AUPRC_average,test_AUPRC_std,test_balanced_accuracy_average,test_balanced_accuracy_std
dataset,,,,,,,,,,,,
A,0.281093,0.240298,0.379570,0.255085,0.443204,0.320742,0.439870,0.287945,0.472825,0.292644,0.652252,0.128655
B,0.353169,0.279759,0.397460,0.277678,0.510088,0.363517,0.443787,0.309523,0.473135,0.314272,0.705559,0.152954
C,0.174801,0.281546,0.251471,0.298361,0.315627,0.362205,0.263973,0.311327,0.343128,0.347088,0.662351,0.254129
D,0.198087,0.217081,0.283299,0.227813,0.338227,0.313166,0.390202,0.295077,0.360113,0.306362,0.658522,0.162238


In [25]:
joined_df.to_csv("outputs/ResMLP_zero_shot_metrics.csv")